In [1]:
SUBTASKS = [2, 3]

In [2]:
from collections import defaultdict
from IPython.display import display
from helper import *
from evaluate import *
import pandas as pd

In [3]:
df_subtask2_train_split = get_performance_tabular(
    "cF1", 2, strategy="train_split")
df_subtask3_train_split = get_performance_tabular(
    "cF1", 3, strategy="train_split")

df_subtasks_train_split = {
    3: df_subtask3_train_split,
    2: df_subtask2_train_split
}

display(df_subtask2_train_split)
display(df_subtask3_train_split)

,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,0.641013,0.646060,0.657267,0.655753
1,English,Laptop,0.634052,0.638545,0.640785,0.639545
2,Japanese,Hotel,0.540414,0.544074,0.544213,0.550660
3,Russian,Restaurant,0.589030,0.599728,0.601384,0.601334
4,Tatar,Restaurant,0.491330,0.496435,0.507477,0.505180
5,Ukrainian,Restaurant,0.571655,0.579969,0.581387,0.584901
6,Chinese,Restaurant,0.728749,0.731102,0.732214,0.733347
7,Chinese,Laptop,0.470715,0.477792,0.483201,0.483983
8,AVG,,0.583370,0.589213,0.593491,0.594338


,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,0.586642,0.595335,0.598888,0.598102
1,English,Laptop,0.383919,0.386321,0.392888,0.393099
2,Japanese,Hotel,0.380169,0.398972,0.400453,0.398643
3,Russian,Restaurant,0.556756,0.563239,0.568803,0.566751
4,Tatar,Restaurant,0.442231,0.456169,0.462152,0.462446
5,Ukrainian,Restaurant,0.524993,0.537554,0.539955,0.545138
6,Chinese,Restaurant,0.702302,0.706533,0.707395,0.706781
7,Chinese,Laptop,0.375443,0.386467,0.395626,0.396089
8,AVG,,0.494057,0.503824,0.508270,0.508381


In [4]:
# results_subtask2 = run_significance_tests(df_subtask2_train_split, 2)
# results_subtask3 = run_significance_tests(df_subtask3_train_split, 3)

In [5]:
display(format_table_parameter_tuning_for_latex(df_subtask2_train_split))
display(format_table_parameter_tuning_for_latex(df_subtask3_train_split))

,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,64.10,64.61,\textbf{65.73},65.58
1,English,Laptop,63.41,63.85,\textbf{64.08},63.95
2,Japanese,Hotel,54.04,54.41,54.42,\textbf{55.07}
3,Russian,Restaurant,58.90,59.97,\textbf{60.14},60.13
4,Tatar,Restaurant,49.13,49.64,\textbf{50.75},50.52
5,Ukrainian,Restaurant,57.17,58.00,58.14,\textbf{58.49}
6,Chinese,Restaurant,72.87,73.11,73.22,\textbf{73.33}
7,Chinese,Laptop,47.07,47.78,48.32,\textbf{48.40}
8,AVG,,58.34,58.92,59.35,\textbf{59.43}


,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,58.66,59.53,\textbf{59.89},59.81
1,English,Laptop,38.39,38.63,39.29,\textbf{39.31}
2,Japanese,Hotel,38.02,39.90,\textbf{40.05},39.86
3,Russian,Restaurant,55.68,56.32,\textbf{56.88},56.68
4,Tatar,Restaurant,44.22,45.62,46.22,\textbf{46.24}
5,Ukrainian,Restaurant,52.50,53.76,54.00,\textbf{54.51}
6,Chinese,Restaurant,70.23,70.65,\textbf{70.74},70.68
7,Chinese,Laptop,37.54,38.65,39.56,\textbf{39.61}
8,AVG,,49.41,50.38,50.83,\textbf{50.84}


In [6]:
# get df_subtask2_train_split values as 1d list
# get df_subtask2_train_split values as 1d list (ohne Language und Domain)
df_values_subtask2 = format_table_parameter_tuning_for_latex(df_subtask2_train_split).drop(columns=["Language", "Domain"]).values.flatten().tolist()
df_values_subtask3 = format_table_parameter_tuning_for_latex(df_subtask3_train_split).drop(columns=["Language", "Domain"]).values.flatten().tolist()

with open(os.path.join("plots", "parameter_optimization_subtask2.txt"), "w") as f:
    f.write(get_tabular_parameter_optimization(df_values_subtask2))

with open(os.path.join("plots", "parameter_optimization_subtask3.txt"), "w") as f:
    f.write(get_tabular_parameter_optimization(df_values_subtask3))

## Export Predictions in Valid Format

In [7]:
strategy_export = "pred_dev"

In [8]:
for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:
        try:
            # best_strategy = get_key_of_best_strategy(
            #     language, domain, df_subtasks_train_split[subtask])
            best_strategy = "sc_15_27b"
            
            num_sc_bs = int(best_strategy.split("_")[1])
            llm_params = best_strategy.split("_")[2]

            predictions = get_performance(language, domain, subtask, strategy_export, llm=f"unsloth/gemma-3-{llm_params}-it-bnb-4bit", num_preds_sc=num_sc_bs)[1]["sc_no_guided"]
            output_dir = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
            os.makedirs(os.path.dirname(output_dir), exist_ok=True)
            with open(output_dir, "w", encoding="utf-8") as f:
                for pred in predictions:
                    f.write(json.dumps(pred, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"Error processing Subtask {subtask} - Language: {language}, Domain: {domain}: {e}")
            continue
        print(
            f"Subtask {subtask} - Language: {language}, Domain: {domain} => Best Strategy: {best_strategy}")

Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Subtask 2 - Language: eng, Domain: restaurant => Best Strategy: sc_15_27b
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Subtask 2 - Language: eng, Domain: laptop => Best Strategy: sc_15_27b
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot 

## Create Tables Dataset Statistics

In [9]:
import pandas as pd

ROWS = ["train", "dev", "test", "test_cross_validation"]
DOMAIN_ORDER = ["restaurant", "laptop", "hotel"]  # Definiere die gewünschte Reihenfolge
SUBTASK_ORDER = [2, 3]  # Erst Subtask 2, dann Subtask 3

records_dataset_statistics = []

for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:

        # train + dev
        for split in ["train", "dev"]:
            count = len(get_dataset(subtask, language, domain, split=split))
            records_dataset_statistics.append((split, domain, subtask, language, count))

        # test (optional)
        try:
            count_test = len(get_dataset(subtask, language, domain, split="test"))
            records_dataset_statistics.append(("test", domain, subtask, language, count_test))
        except:
            pass

        # test_cross_validation (als ganze Zahl)
        train_size = len(get_dataset(subtask, language, domain, split="train"))
        records_dataset_statistics.append((
            "test_cross_validation",
            domain,
            subtask,
            language,
            str(int(train_size * 0.2))
        ))

df_dataset_statistics = pd.DataFrame(
    records_dataset_statistics,
    columns=["split", "domain", "subtask", "language", "count"]
)

# Konvertiere domain zu Categorical mit gewünschter Reihenfolge
df_dataset_statistics["domain"] = pd.Categorical(
    df_dataset_statistics["domain"], 
    categories=DOMAIN_ORDER, 
    ordered=True
)

df_dataset_statistics = (
    df_dataset_statistics
        .pivot(index=["split", "domain"],
               columns=["subtask", "language"],
               values="count")
        .sort_index(level=["split", "domain"], key=lambda x: x.map({s: i for i, s in enumerate(ROWS)} if x.name == "split" else {d: i for i, d in enumerate(DOMAIN_ORDER)}))
)

# Spalten nach gewünschter Subtask-Reihenfolge sortieren (erst 2, dann 3)
df_dataset_statistics = df_dataset_statistics.reindex(
    columns=sorted(df_dataset_statistics.columns, key=lambda x: (SUBTASK_ORDER.index(x[0]), x[1]))
)

df_dataset_statistics = df_dataset_statistics.applymap(
    lambda x: f"{int(x):,}" if pd.notna(x) else "-"
)

# get values from left to right from top to bottom as 1D list
values_list_dataset_statistics = df_dataset_statistics.values.flatten().tolist()
df_dataset_statistics

/tmp/ipykernel_2289637/2424028441.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dataset_statistics = df_dataset_statistics.applymap(


subtask                               2                                     \
language                            eng    jpn    rus    tat    ukr    zho   
split                 domain                                                 
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050   
                      laptop      4,076      -      -      -      -  3,490   
                      hotel           -  1,600      -      -      -      -   
dev                   restaurant    200      -     48     48     48    300   
                      laptop        200      -      -      -      -    300   
                      hotel           -    200      -      -      -      -   
test_cross_validation restaurant    456      -    248    248    248  1,210   
                      laptop        815      -      -      -      -    698   
                      hotel           -    320      -      -      -      -   

subtask                               3                                     
language                            eng    jpn    rus    tat    ukr    zho  
split                 domain                                                
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050  
                      laptop      4,076      -      -      -      -  3,490  
                      hotel           -  1,600      -      -      -      -  
dev                   restaurant    200      -     48     48     48    300  
                      laptop        200      -      -      -      -    300  
                      hotel           -    200      -      -      -      -  
test_cross_validation restaurant    456      -    248    248    248  1,210  
                      laptop        815      -      -      -      -    698  
                      hotel           -    320      -      -      -      -

In [10]:
with open("plots/muster/dataset.txt", "r", encoding="utf-8") as f:
    dataset_muster = f.read()

# gehe von xxxx zu xxxx und trage die Werte ein
for value in values_list_dataset_statistics:
    dataset_muster = dataset_muster.replace("xxxx", value, 1)

with open("plots/dataset_statistics.txt", "w", encoding="utf-8") as f:
    f.write(dataset_muster)